In [5]:
from bs4 import BeautifulSoup
import pandas as pd
import requests

url= 'https://scholar.google.com/scholar'

#set headers
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) \
    AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36' 
    #english
}

params = {
    #english
    'hl': 'en',

    'start': 0,
    'q': 'heat stress guidlines interventions mitigation strategies' 
}

In [6]:
##GET TITLE/LINK/AUTHORS OF N*10 ARTICLES

#loop through pages for n pages
n = 1

from tqdm.notebook import tqdm

#store results
results = pd.DataFrame(columns=['title', 'link', 'author'])

for i in tqdm(range(n)):
    params['start'] = i*10
    response = requests.get(url, headers=headers, params=params)
    soup = BeautifulSoup(response.text, 'html.parser')
    articles = soup.find_all('div', class_='gs_ri')

    #get titles
    titles = [article.find('h3', class_='gs_rt').text for article in articles]
    #get links
    links = [article.find('h3', class_='gs_rt').find('a')['href'] for article in articles]
    #get authors
    authors = [article.find('div', class_='gs_a').text for article in articles]
    
    #add to results
    results = results._append(pd.DataFrame({'title': titles, 'link': links, 'author': authors}))

  0%|          | 0/1 [00:00<?, ?it/s]

In [7]:
results.to_csv('article_info.csv')

In [8]:
## BASIC CODE TO SCRAPE ABSTRACT 

import requests
from bs4 import BeautifulSoup

def scrape_abstract(url):

    # YOUR CODE HERE
    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')

        abstract = soup.find('blockquote', class_='abstract').text.strip()
    
    return abstract

In [9]:
link = 'https://www.sciencedirect.com/science/article/pii/S0921800919302976'

response = requests.get(link, headers=headers)

print(response)

<Response [403]>


In [10]:
## CODE TO DOWNLOAD 1 LINK

from selenium import webdriver

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

import time
import requests
from urllib.parse import urlparse

main_page_url = 'https://sci-hub.ru/'
url_to_search = 'https://www.sciencedirect.com/science/article/pii/S0921800919302976'
driver = webdriver.Chrome()

try:
    # Open page
    driver.get(main_page_url)
    time.sleep(2)  # Wait for the page to load (you can decrease this time if need be)

    #doi_input = driver.find_element()
    # Find search bar -> input URL
    search_bar = driver.find_element(by=By.ID, value='request')
    search_bar.send_keys(url_to_search)
    search_bar.send_keys(Keys.RETURN)  # To submit the search query
    time.sleep(1)  # Wait for a bit again

    # Find download link
    download_button = driver.find_element(by=By.XPATH, value="//button[@onclick]")
    onclick_attribute = download_button.get_attribute('onclick')

    # Extract URL from the onclick attribute
    # Assuming the format is location.href='URL'
    pdf_url = onclick_attribute.split("'")[1]
    
    download_link = 'https://' + urlparse(main_page_url).hostname + pdf_url
  
    # Download the PDF using requests
    response = requests.get(download_link)

    with open('pdf.pdf', 'wb') as file:
        file.write(response.content)

except Exception as e:
    # Handle the exception, print an error message, etc.
    print(f"An error occurred: {str(e)}")
finally:
    driver.quit()

In [11]:
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

import time
import requests
import os
import random

def pdf_download(url_to_search, file_name):
    driver = webdriver.Chrome()  # initialize web driver
    main_page_url = 'https://sci-hub.ru/'  # open sci-hub
    driver.get(main_page_url)

    try:
        # Wait until the search bar with id="request" is present
        search_bar = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, 'request'))
        )
        search_bar.send_keys(url_to_search)
        search_bar.send_keys(Keys.RETURN)  # Submit the search query
    except Exception as e:
        print(f"Could not find search bar for {url_to_search}. Error: {e}")
        driver.quit()
        return None

    time.sleep(random.uniform(4, 6))

    try:
        # Find download button
        download_button = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//button[@onclick]"))
        )
        onclick_attribute = download_button.get_attribute('onclick')
    except Exception as e:
        print(f"Could not find download button for {url_to_search}. Error: {e}")
        driver.quit()
        return url_to_search  # Mark this URL as problematic

    try:
        # Extract URL from onclick attribute
        pdf_url = onclick_attribute.split("'")[1]
        download_link = 'https://sci-hub.ru' + pdf_url

        # Ensure 'pdfs' folder exists
        os.makedirs('pdfs', exist_ok=True)

        # Download and save the PDF
        response = requests.get(download_link)
        with open(f'pdfs/{file_name}.pdf', 'wb') as file:
            file.write(response.content)

    except Exception as e:
        print(f"Failed to download or save PDF for {url_to_search}. Error: {e}")
        driver.quit()
        return url_to_search

    driver.quit()
    return None  # Success


In [8]:
##to call on all links !!! DO NOT RERUN, IT WILL SET LIST BACK TO NONE

import pandas as pd
import os

data = pd.read_csv('article_info.csv')
title = data['title']
links = data['link']
author = ['author'] 


def file_exists(file_path):
    return os.path.exists(file_path)

list_error_links = []

In [9]:
#function to call on all links
os.makedirs('pdfs', exist_ok=True)

for index, row in data.iterrows():
    pdf_file_path = f'pdfs/{index}.pdf'
    if file_exists(pdf_file_path) is True:
        print("PDF already downloaded")
    else:
        if row['link'] in list_error_links:
            print('error already encountered')
        else:
            url_returned = pdf_download(row['link'], index)  #(links[index], index)
            if url_returned is not None:
                list_error_links.append(url_returned)
                print(f'ERROR FOR {url_returned}')

PDF already downloaded
Could not find download button for https://www.jstage.jst.go.jp/article/indhealth/51/1/51_2012-0089/_article/-char/ja/. Error: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF629184C25+3179557]
	(No symbol) [0x00007FF628DE88A0]
	(No symbol) [0x00007FF628C791CA]
	(No symbol) [0x00007FF628CCFA67]
	(No symbol) [0x00007FF628CCFC9C]
	(No symbol) [0x00007FF628D23627]
	(No symbol) [0x00007FF628CF7C6F]
	(No symbol) [0x00007FF628D202F3]
	(No symbol) [0x00007FF628CF7A03]
	(No symbol) [0x00007FF628CC06D0]
	(No symbol) [0x00007FF628CC1983]
	GetHandleVerifier [0x00007FF6291E67CD+3579853]
	GetHandleVerifier [0x00007FF6291FD1D2+3672530]
	GetHandleVerifier [0x00007FF6291F2153+3627347]
	GetHandleVerifier [0x00007FF628F5092A+868650]
	(No symbol) [0x00007FF628DF2FFF]
	(No symbol) [0x00007FF628DEF4A4]
	(No symbol) [0x00007FF628DEF646]
	(No symbol) [0x00007FF628DDEAA9]
	BaseThreadInitThunk [0x00007FFD3EE2259D+29]
	RtlUserThreadStart [0x00007FFD3F9AAF38+40]

ERROR FOR https://www.j

In [10]:
%pip install pypdf

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: C:\Users\Gur Levy\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [ ]:
from pypdf import PdfReader
from os import listdir
from os.path import isfile, join

mypath = 'pdfs'  
pdf_folder_content = [f for f in listdir(mypath) if isfile(join(mypath, f)) and f.lower().endswith('.pdf')]

all_pdf_texts = {}

for pdf in pdf_folder_content:
    print(f'Analyzing {pdf}...')

    try:
        reader = PdfReader(join(mypath, pdf))
        n_pages = len(reader.pages)
        print(f'{pdf} has {n_pages} pages!')

        all_text = []
        for i in range(n_pages):  
            page_text = reader.pages[i].extract_text()
            all_text.append(page_text)

        result_string = ' '.join(filter(None, all_text))  # remove None entries
        all_pdf_texts[pdf] = result_string

    except Exception as e:
        print(f"Skipping {pdf} — couldn't read PDF. Error: {e}")


Analyzing 0.pdf...
0.pdf has 16 pages!


invalid pdf header: b'<!DOC'
EOF marker not found
invalid pdf header: b'<!DOC'
EOF marker not found
invalid pdf header: b'<!DOC'
EOF marker not found
invalid pdf header: b'<!DOC'
EOF marker not found
invalid pdf header: b'<!DOC'
EOF marker not found
invalid pdf header: b'<!DOC'
EOF marker not found


Analyzing 10.pdf...
Skipping 10.pdf — couldn't read PDF. Error: Stream has ended unexpectedly
Analyzing 11.pdf...
Skipping 11.pdf — couldn't read PDF. Error: Stream has ended unexpectedly
Analyzing 14.pdf...
Skipping 14.pdf — couldn't read PDF. Error: Stream has ended unexpectedly
Analyzing 15.pdf...
Skipping 15.pdf — couldn't read PDF. Error: Stream has ended unexpectedly
Analyzing 16.pdf...
Skipping 16.pdf — couldn't read PDF. Error: Stream has ended unexpectedly
Analyzing 17.pdf...
Skipping 17.pdf — couldn't read PDF. Error: Stream has ended unexpectedly
Analyzing 18.pdf...
18.pdf has 16 pages!


invalid pdf header: b'<!DOC'
EOF marker not found
invalid pdf header: b'<!DOC'
EOF marker not found
invalid pdf header: b'<!DOC'
EOF marker not found
invalid pdf header: b'<!DOC'
EOF marker not found
invalid pdf header: b'<!DOC'
EOF marker not found
invalid pdf header: b'<!DOC'
EOF marker not found
invalid pdf header: b'<!DOC'


Analyzing 2.pdf...
Skipping 2.pdf — couldn't read PDF. Error: Stream has ended unexpectedly
Analyzing 20.pdf...
Skipping 20.pdf — couldn't read PDF. Error: Stream has ended unexpectedly
Analyzing 22.pdf...
Skipping 22.pdf — couldn't read PDF. Error: Stream has ended unexpectedly
Analyzing 23.pdf...
Skipping 23.pdf — couldn't read PDF. Error: Stream has ended unexpectedly
Analyzing 25.pdf...
Skipping 25.pdf — couldn't read PDF. Error: Stream has ended unexpectedly
Analyzing 28.pdf...
Skipping 28.pdf — couldn't read PDF. Error: Stream has ended unexpectedly
Analyzing 3.pdf...


...

Analyzing 86.pdf...
Skipping 86.pdf — couldn't read PDF. Error: Stream has ended unexpectedly
Analyzing 87.pdf...
Skipping 87.pdf — couldn't read PDF. Error: Stream has ended unexpectedly
Analyzing 88.pdf...
Skipping 88.pdf — couldn't read PDF. Error: Stream has ended unexpectedly
Analyzing 9.pdf...
Skipping 9.pdf — couldn't read PDF. Error: Stream has ended unexpectedly
Analyzing 90.pdf...
Skipping 90.pdf — couldn't read PDF. Error: Stream has ended unexpectedly
Analyzing 91.pdf...
Skipping 91.pdf — couldn't read PDF. Error: Stream has ended unexpectedly
Analyzing 92.pdf...
Skipping 92.pdf — couldn't read PDF. Error: Stream has ended unexpectedly
Analyzing 93.pdf...
93.pdf has 17 pages!
Analyzing 94.pdf...
94.pdf has 14 pages!


invalid pdf header: b'<!DOC'
EOF marker not found


Analyzing 96.pdf...
Skipping 96.pdf — couldn't read PDF. Error: Stream has ended unexpectedly
Analyzing 97.pdf...
97.pdf has 68 pages!


invalid pdf header: b'<!DOC'
EOF marker not found
invalid pdf header: b'<!DOC'
EOF marker not found


Analyzing 98.pdf...
Skipping 98.pdf — couldn't read PDF. Error: Stream has ended unexpectedly
Analyzing 99.pdf...
Skipping 99.pdf — couldn't read PDF. Error: Stream has ended unexpectedly


In [ ]:
##DO NOT RUN THIE CODE BELOW

In [1]:
##TOPIC MODELLING
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
import re


from nltk.stem import WordNetLemmatizer
from gensim import corpora
from gensim.models.ldamodel import LdaModel

lemmatizer = WordNetLemmatizer()

additional_stopwords = ['et', 'al','b', 'fw', 'e'] 

def preprocess_text(text):
    #YOUR CODE HERE

    # removes special characters and punctuation (STEP 1: Text cleaning)
    text = re.sub(r'[^A-Za-z\s]', '', text) 
    # remove additional white spaces (STEP 1: Text cleaning)
    text = re.sub(r'\s+', ' ', text).strip() 

    #put text in lowercases (STEP 2: Case normalization)
    text = text.lower()

    ###STEP 3: tokenization - FIX THIS 
    tokens = word_tokenize(text)

    #STEP 4: removing stopwords:
    stopwords_list = set(stopwords.words('english')).union(additional_stopwords)
    tokens = [token for token in tokens if token not in stopwords_list]

    #STEP 5: lemmeatization: 
    lemmatized_words = [lemmatizer.lemmatize(token) for token in tokens]
    
    return lemmatized_words

In [2]:
processed_text = [preprocess_text(text) for text in all_pdf_texts]

corpus = corpora.Dictionary(processed_text)
doc_term_matrix = [corpus.doc2bow(text) for text in processed_text]

num_topics = 45

lda_model = LdaModel(doc_term_matrix, id2word = corpus, num_topics = num_topics)


NameError: name 'all_pdf_texts' is not defined

In [3]:
from collections import Counter

document_topics = [lda_model.get_document_topics(doc) for doc in doc_term_matrix]
all_topics = [topic[0] for topics in document_topics for topic in topics]

topics_nb = Counter(all_topics)

most_nb_topics = [topic for topic, count in topics_nb.most_common()]
print("The most common topics and the number of instances are:")
for topic, x in topics_nb.most_common():
    print(f"Topic {topic +1}: Count {x}")
for topic in most_nb_topics:
    topics_words = lda_model.show_topic(topic)
    most_words = [word for word, _ in topics_words]
    print(f"Most used words in Topic {topic +1}: {', '.join(most_words)}")

NameError: name 'doc_term_matrix' is not defined

In [4]:
from gensim.models.coherencemodel import CoherenceModel

# Function to compute coherence score
def compute_coherence(dictionary, corpus, texts, limit, start=2, step=3):
    # YOUR CODE HERE
    coherence_values = []
    for num_topics in range(start, limit, step):
        lda_model = LdaModel(corpus = corpus, id2word = dictionary, num_topics = num_topics, random_state = 100)

        coherence_model_lda = CoherenceModel(model = lda_model, texts = texts, dictionary = dictionary, coherence = 'c_v')
        coherence_values.append(coherence_model_lda.get_coherence())
    
    return coherence_values

coherence_values = compute_coherence(dictionary=corpus, corpus= doc_term_matrix, texts=processed_text, limit=50, start = 5, step=5)

optimal_nb_topics = (coherence_values.index(max(coherence_values)) + 1) * 5
print(f"the optimal number of topic for this model is: {optimal_nb_topics}")

NameError: name 'corpus' is not defined